# IMPORTING THE LIBRARIES 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [3]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store CSV file names
    csv_file_names = []

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            # Read CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Extract intensity column and store it in the list
            intensity_dfs.append(df.iloc[:, 1])  # Assuming intensity is in the second column
            
            # Store CSV file name
            csv_file_names.append(os.path.splitext(file_name)[0])

    # Read the wavelength column from the first CSV file
    wavelength_df = pd.read_csv(os.path.join(folder_path, os.listdir(folder_path)[1]), usecols=[0])

    # Concatenate wavelength column with intensity columns
    result_df = pd.concat([wavelength_df] + intensity_dfs, axis=1)

    # Rename the columns with CSV file names
    column_names = ['wavelength'] + [f'{csv_file_names[i]}' for i in range(len(intensity_dfs))]

    result_df.columns = column_names
    # result_df.reset_index(drop=True, inplace=True)

    return result_df

In [4]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.airpls(df[col],lam=0.1)
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

Loading the dataframe.

In [5]:
CR210LA_Raw_df = load_data('CR210LA')
###################################################
CR240LA_Raw_df = load_data('CR240LA')
###################################################
CR300LA_Raw_df = load_data('CR300LA')
###################################################
CR440Y_Raw_df = load_data('CR440Y')
###################################################
CR570CP_Raw_df = load_data('CR570CP')
###################################################
CR700Y_Raw_df = load_data('CR700Y')
###################################################
HR660Y_Raw_df = load_data('HR660Y')
# ###################################################
CR1000Y_Raw_df = load_data('CR1000Y')

In [6]:
CR210LA_BaselineCorrected_df = CR210LA_Raw_df

CR240LA_BaselineCorrected_df = CR240LA_Raw_df

CR300LA_BaselineCorrected_df = CR300LA_Raw_df

CR440Y_BaselineCorrected_df = CR440Y_Raw_df

CR570CP_BaselineCorrected_df = CR570CP_Raw_df

CR700Y_BaselineCorrected_df = CR700Y_Raw_df

HR660Y_BaselineCorrected_df = HR660Y_Raw_df

CR1000Y_BaselineCorrected_df = CR1000Y_Raw_df

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [7]:
Wavelength_Min = 202.15
Wavelength_Max = 203.1

Element_name = 'Mo'

CR210LA_Select_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_df


,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,3364.614875,3864.423606,2452.816479,3876.268310,3759.857255,2140.434378,2127.833692,2234.583455,2182.162847,2043.497661,2135.632526,2432.109944
666,202.200000,3824.006769,4524.889663,2927.468415,4573.492982,4359.126010,2535.740775,2526.233444,2653.650614,2603.911206,2456.339939,2530.314392,2873.306398
667,202.233333,4458.673998,5297.455347,3616.251612,5357.443087,5047.820574,3129.888445,3121.314779,3278.335265,3222.773666,3042.964378,3123.961542,3530.601359
668,202.266667,5372.917206,6293.190532,4440.062945,6328.524379,5941.789375,3836.634334,3820.251748,4020.992865,3951.366120,3716.957912,3824.987836,4315.705609
669,202.300000,6702.543963,7815.527173,5376.546303,7808.883044,7353.715483,4627.204777,4592.620231,4854.665221,4765.229960,4455.099202,4601.814574,5199.266044
670,202.333333,8583.105057,10173.115761,6718.370382,10128.676092,9602.626307,5792.036631,5754.459237,6089.350684,5989.067083,5584.527700,5756.159765,6479.342402
671,202.366667,11060.552257,13321.299837,8865.998524,13267.754990,12674.702452,7730.794752,7740.360898,8150.347385,8067.055259,7552.269920,7703.789270,8566.028510
672,202.400000,14080.344118,16819.157997,11991.962970,16813.072606,16182.807440,10642.614188,10761.084542,11234.967715,11211.001322,10585.495159,10645.985913,11647.062674
673,202.433333,17564.371969,20213.013926,15445.082828,20335.011359,19722.411677,14001.895985,14218.446901,14716.589803,14792.923677,14116.787170,14008.879004,15106.597727
674,202.466667,21157.240023,23174.579366,18389.232580,23468.627484,22925.376201,17120.018677,17332.440560,17783.477217,17996.103226,17400.019480,17044.360212,18148.283205


In [8]:
CR210LA_Select_Raw_df = CR210LA_Raw_df[(CR210LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR210LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_Raw_df = CR240LA_Raw_df[(CR240LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR240LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_Raw_df = CR300LA_Raw_df[(CR300LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR300LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_Raw_df = CR440Y_Raw_df[(CR440Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR440Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_Raw_df = CR570CP_Raw_df[(CR570CP_Raw_df['wavelength'] >= Wavelength_Min) & (CR570CP_Raw_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_Raw_df = CR700Y_Raw_df[(CR700Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR700Y_Raw_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_Raw_df = HR660Y_Raw_df[(HR660Y_Raw_df['wavelength'] >= Wavelength_Min) & (HR660Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_Raw_df = CR1000Y_Raw_df[(CR1000Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR1000Y_Raw_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_Raw_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,3364.614875,3864.423606,2452.816479,3876.268310,3759.857255,2140.434378,2127.833692,2234.583455,2182.162847,2043.497661,2135.632526,2432.109944
666,202.200000,3824.006769,4524.889663,2927.468415,4573.492982,4359.126010,2535.740775,2526.233444,2653.650614,2603.911206,2456.339939,2530.314392,2873.306398
667,202.233333,4458.673998,5297.455347,3616.251612,5357.443087,5047.820574,3129.888445,3121.314779,3278.335265,3222.773666,3042.964378,3123.961542,3530.601359
668,202.266667,5372.917206,6293.190532,4440.062945,6328.524379,5941.789375,3836.634334,3820.251748,4020.992865,3951.366120,3716.957912,3824.987836,4315.705609
669,202.300000,6702.543963,7815.527173,5376.546303,7808.883044,7353.715483,4627.204777,4592.620231,4854.665221,4765.229960,4455.099202,4601.814574,5199.266044
670,202.333333,8583.105057,10173.115761,6718.370382,10128.676092,9602.626307,5792.036631,5754.459237,6089.350684,5989.067083,5584.527700,5756.159765,6479.342402
671,202.366667,11060.552257,13321.299837,8865.998524,13267.754990,12674.702452,7730.794752,7740.360898,8150.347385,8067.055259,7552.269920,7703.789270,8566.028510
672,202.400000,14080.344118,16819.157997,11991.962970,16813.072606,16182.807440,10642.614188,10761.084542,11234.967715,11211.001322,10585.495159,10645.985913,11647.062674
673,202.433333,17564.371969,20213.013926,15445.082828,20335.011359,19722.411677,14001.895985,14218.446901,14716.589803,14792.923677,14116.787170,14008.879004,15106.597727
674,202.466667,21157.240023,23174.579366,18389.232580,23468.627484,22925.376201,17120.018677,17332.440560,17783.477217,17996.103226,17400.019480,17044.360212,18148.283205


Lets plot a line plot ,to get a better picture 

In [9]:
Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11, line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_11, line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_11, line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_df_Plot.width = 600
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.

In [10]:
Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Baseline_Correction_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="green" )
Baseline_Correction_Plot.line(CR210LA_Select_Raw_df.wavelength,CR210LA_Select_Raw_df.CR210LA_1 , line_width =2 , color = "red")
Baseline_Correction_Plot.width = 600
Baseline_Correction_Plot.height = 500
show(Baseline_Correction_Plot)

2) Normalization - Its usually noticed that , for a measurement of a similar sample , the spectra obtained for them are quite different , but there could me many reasons for it , like laser energy fluctuations , material surface , even though is homegnous isnt same overall the surface

   For Instance , you could see the plot below , Though this are the plots from the same sample CR210LA , measured on 12 different Areas , its quite visible , for some  spectras , the peak heights or Intensities are not same.


In [11]:

Before_Normalization = figure(title = 'Before Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12 , line_width = 2, color ="red" )


Before_Normalization.width = 600
Before_Normalization.height = 500
show(Before_Normalization)

Now , its important the understand this chnages in Peaks , are common not just to a single peaks , but to the entire spectra. If theres a change in intensity of a peak , its not just for a single peak , but it applies to the entire spectrum proportionally. 

Normalization by peak of an Matrix Element i.e Fe 238.20 (237.9 ~ 238.7) , 263.10 (262.9 ~ 263.4)

Lets try to focus now on analysis for peak of Matrix Element

In [12]:
Matrix_Wavelength_Min = 262.9
Matrix_Wavelength_Max = 263.4

CR210LA_Matrix_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR240LA_Matrix_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR300LA_Matrix_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR440Y_Matrix_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR570CP_Matrix_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR700Y_Matrix_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
HR660Y_Matrix_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR1000Y_Matrix_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]

CR210LA_Matrix_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
2487,262.900000,4837.239661,2890.524843,2806.221541,1972.105107,3362.020617,3430.222580,3336.461647,3045.610684,3006.941429,2397.305049,3574.642021,3322.774215
2488,262.933333,3858.095053,2007.470809,2997.578234,1276.561036,2546.568322,3628.344706,3522.650443,3253.566277,3191.822673,2431.437508,3734.662130,3286.920716
2489,262.966667,5890.431865,3584.409197,4549.822425,2213.051646,4240.959231,5207.277569,5163.010963,4821.534564,4784.486295,3601.703971,5426.559016,4599.751121
2490,263.000000,11539.901504,8275.813898,7060.001122,5272.049724,9030.834997,7795.671624,7863.594916,7352.460742,7392.546407,5685.782566,8268.308163,7071.833508
2491,263.033333,18192.166254,13813.596846,9978.377543,8988.179265,14624.825375,10876.669172,11076.226103,10303.527765,10473.674795,8344.676369,11717.032409,10378.969458
2492,263.066667,22084.894478,16887.774737,12412.813312,11198.632465,17705.974471,13510.273579,13821.766394,12772.132914,13088.726037,10779.589549,14724.483441,13531.212976
2493,263.100000,21032.733086,15640.850727,13046.998398,10652.129259,16412.372951,14232.298055,14587.196277,13409.966176,13806.972267,11622.121256,15616.352419,14713.881097
2494,263.133333,16752.047636,11808.983135,10748.636788,8352.773505,12482.043844,11776.209668,12064.271684,11056.010995,11393.362199,9675.559222,12939.642946,12329.476564
2495,263.166667,11525.757655,7640.777387,6535.763552,5630.205967,8167.093209,7228.765854,7376.987247,6746.783708,6925.208524,5859.937709,7901.188565,7524.374497
2496,263.200000,6872.636899,4403.211740,2685.258532,3314.297509,4758.240346,3054.385058,3073.216457,2805.278268,2833.525848,2334.474715,3264.937928,3051.265171


In [13]:
Selected_Matrix_Plot = figure(title = 'Selected Matrix Peak' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_11, line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_11, line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_11, line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_Matrix_Plot.width = 600
Selected_Matrix_Plot.height = 500
show(Selected_Matrix_Plot)

In [14]:

def normalize_data(matrix_df, baseline_df):
    normalized_df = pd.DataFrame() #crete a df
    normalized_df['wavelength'] = baseline_df['wavelength'] #add first column as wavelength

    for sample_col in matrix_df.columns[1:]:
        peaks, _ = fp(matrix_df[sample_col], prominence=1000) #finds peak from matrix_df
        return_intensities = matrix_df[sample_col].iloc[peaks] #gets the intensity for that matrix peak
        # return_intensities = max(matrix_df[sample_col]]      #Normalization by Maximum Intensity
        corresponding_col = sample_col
        normalized_df[sample_col] = baseline_df[corresponding_col] / return_intensities.values[0] #create a columns in normalized_df , by dividing the baseline_df by the intensities of the matrix peak 
    
    return normalized_df


In [15]:
# normalized_df = pd.DataFrame()
# normalized_df['wavelength'] = CR210LA_BaselineCorrected_df['wavelength']

# for sample_col in CR210LA_Matrix_df.columns[1:]:
#         peaks, _ = fp(CR210LA_Matrix_df[sample_col], prominence=1000 )
#         return_intensities = CR210LA_Matrix_df[sample_col].iloc[peaks]
#         correspondin_col = sample_col
#         print(return_intensities)
#         print(correspondin_col)
#         normalized_df[sample_col] = CR210LA_BaselineCorrected_df[correspondin_col] / return_intensities.values[0] 

# print (normalized_df)

In [16]:
CR210LA_Normalized_df = normalize_data (matrix_df=CR210LA_Matrix_df , baseline_df=CR210LA_BaselineCorrected_df)
CR240LA_Normalized_df = normalize_data (matrix_df=CR240LA_Matrix_df , baseline_df=CR240LA_BaselineCorrected_df)
CR300LA_Normalized_df = normalize_data (matrix_df=CR300LA_Matrix_df , baseline_df=CR300LA_BaselineCorrected_df)
CR440Y_Normalized_df = normalize_data (matrix_df=CR440Y_Matrix_df , baseline_df=CR440Y_BaselineCorrected_df)
CR570CP_Normalized_df = normalize_data (matrix_df=CR570CP_Matrix_df , baseline_df=CR570CP_BaselineCorrected_df)
CR700Y_Normalized_df = normalize_data (matrix_df=CR700Y_Matrix_df , baseline_df=CR700Y_BaselineCorrected_df)
HR660Y_Normalized_df = normalize_data (matrix_df=HR660Y_Matrix_df , baseline_df=HR660Y_BaselineCorrected_df)
CR1000Y_Normalized_df = normalize_data (matrix_df=CR1000Y_Matrix_df , baseline_df=CR1000Y_BaselineCorrected_df)
# print (CR210LA_Normalized_df.iloc[2493])

In [17]:
CR210LA_Processed_Select_df = CR210LA_Normalized_df[(CR210LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR210LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR240LA_Processed_Select_df = CR240LA_Normalized_df[(CR240LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR240LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR300LA_Processed_Select_df = CR300LA_Normalized_df[(CR300LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR300LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR440Y_Processed_Select_df = CR440Y_Normalized_df[(CR440Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR440Y_Normalized_df['wavelength'] <= Wavelength_Max)]
CR570CP_Processed_Select_df = CR570CP_Normalized_df[(CR570CP_Normalized_df['wavelength'] >= Wavelength_Min) & (CR570CP_Normalized_df['wavelength'] <= Wavelength_Max)]
CR700Y_Processed_Select_df = CR700Y_Normalized_df[(CR700Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR700Y_Normalized_df['wavelength'] <= Wavelength_Max)]
HR660Y_Processed_Select_df = HR660Y_Normalized_df[(HR660Y_Normalized_df['wavelength'] >= Wavelength_Min) & (HR660Y_Normalized_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Processed_Select_df = CR1000Y_Normalized_df[(CR1000Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR1000Y_Normalized_df['wavelength'] <= Wavelength_Max)]

CR210LA_Processed_Select_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,0.152349,0.228830,0.187999,0.346138,0.212350,0.150393,0.145870,0.166636,0.158048,0.175828,0.136756,0.165294
666,202.200000,0.173150,0.267939,0.224379,0.408397,0.246195,0.178168,0.173182,0.197886,0.188594,0.211350,0.162030,0.195279
667,202.233333,0.201888,0.313686,0.277171,0.478402,0.285091,0.219914,0.213976,0.244470,0.233416,0.261825,0.200044,0.239950
668,202.266667,0.243285,0.372648,0.340313,0.565116,0.335581,0.269572,0.261891,0.299851,0.286186,0.319818,0.244935,0.293308
669,202.300000,0.303490,0.462792,0.412091,0.697307,0.415324,0.325120,0.314839,0.362019,0.345132,0.383329,0.294679,0.353358
670,202.333333,0.388641,0.602395,0.514936,0.904457,0.542338,0.406964,0.394487,0.454091,0.433771,0.480508,0.368598,0.440356
671,202.366667,0.500820,0.788813,0.679543,1.184766,0.715843,0.543187,0.530627,0.607783,0.584274,0.649819,0.493316,0.582173
672,202.400000,0.637555,0.995937,0.919136,1.501351,0.913974,0.747779,0.737708,0.837807,0.811981,0.910806,0.681720,0.791570
673,202.433333,0.795312,1.196902,1.183804,1.815848,1.113885,0.983811,0.974721,1.097437,1.071410,1.214648,0.897065,1.026690
674,202.466667,0.957996,1.372270,1.409461,2.095669,1.294782,1.202899,1.188195,1.326139,1.303407,1.497147,1.091443,1.233412


In [18]:

After_Normalization = figure(title = 'After Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_1 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_2 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_3 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_4 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_5 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_6 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_7 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_8 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_9 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_10 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_11 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_12 , line_width = 2, color ="red" )


After_Normalization.width = 600
After_Normalization.height = 500
show(After_Normalization)

# OUTLIER REMOVAL 

As you see from the Pic above , even after normalizing the Spectra , there are few variations in the Intensities , Before passing them to regression step , the important step would be to do Outlier Removal of the Intensities

In [19]:
def clean_intensities(processed_df, threshold):
    """
    Cleans intensities by removing outliers based on z-score method.

    Args:
    processed_df (DataFrame): DataFrame containing intensity data.
    threshold (float): Threshold for z-score.

    Returns:
    list: List containing cleaned intensities.
    """
    all_intensities = []

    for col in processed_df.columns[1:]:
        peak, _ = fp(processed_df[col], prominence=0.5)
        all_intensities.extend(processed_df[col].iloc[peak])

    mean_intensities = np.mean(all_intensities)
    std_intensities = np.std(all_intensities)
    z_scores = np.abs((all_intensities - mean_intensities) / std_intensities)

    cleaned_intensities = [intensity for intensity, z_score in zip(all_intensities, z_scores) if z_score < threshold]

    return cleaned_intensities


In [20]:
CR210LA_Selected_Intensities = clean_intensities(processed_df=CR210LA_Processed_Select_df , threshold=2)
CR240LA_Selected_Intensities = clean_intensities(processed_df=CR240LA_Processed_Select_df , threshold=2)
CR300LA_Selected_Intensities = clean_intensities(processed_df=CR300LA_Processed_Select_df , threshold=2)
CR440Y_Selected_Intensities = clean_intensities(processed_df=CR440Y_Processed_Select_df , threshold=2)
CR570CP_Selected_Intensities = clean_intensities(processed_df=CR570CP_Processed_Select_df , threshold=2)
CR700Y_Selected_Intensities = clean_intensities(processed_df=CR700Y_Processed_Select_df , threshold=2)
HR660Y_Selected_Intensities = clean_intensities(processed_df=HR660Y_Processed_Select_df , threshold=2)
CR1000Y_Selected_Intensities = clean_intensities(processed_df=CR1000Y_Processed_Select_df , threshold=2)

print (CR300LA_Selected_Intensities)

[1.0461075653307756, 1.2694798154043536, 1.3639455155161089, 1.2517554500656811, 1.1411207293354253, 1.2346191953618906, 1.0136308185673855, 1.083365710711073, 1.1900996424675736, 1.1702540633144405, 1.2203880566066796]


In [21]:
# all_intensities = []

# for col in CR300LA_Processed_Select_df.columns[1:]:
#         peak, _ = fp(CR300LA_Processed_Select_df[col],prominence=0.5)
#         all_intensities.extend(CR300LA_Processed_Select_df[col].iloc[peak])
    
# print(all_intensities)
        
# mean_intensities = np.mean(all_intensities)
# std_intensities = np.std(all_intensities)
# z_scores = np.abs((all_intensities - mean_intensities) / std_intensities)

# cleaned_intensities = [intensity for intensity, z_score in zip(all_intensities, z_scores) if z_score < 2]
# print(cleaned_intensities)

# REGRESSION

At the last , now we have list we have selected intensities for each sample , also we have the concentration values per element of each sample , we can now do the regression

In [22]:
concentration_data = pd.read_csv(r'Concentration.csv', skiprows=0)
concentration_data

,wavelength,CR210LA,CR240LA,CR300LA,CR570CP,CR440Y,CR700Y,CR1000Y,HR660Y
0,Fe,99.00,99.00,99.00,97.0,97.0,96.0,98.0,97.0
1,Cu,0.10,0.10,0.10,0.1,0.1,0.1,0.1,0.1
2,Mo,0.00,0.05,0.05,0.5,0.6,0.5,0.3,0.5
3,Ni,0.05,0.05,0.10,0.7,0.1,0.0,0.0,0.1
4,Al,0.00,0.10,0.05,0.5,0.8,0.5,0.2,0.5
5,Si,0.20,0.20,0.30,0.5,0.4,0.5,0.2,0.5
6,Cr,0.00,0.05,0.05,0.5,0.7,0.5,0.4,0.5
7,Mn,0.30,0.40,0.60,1.2,1.2,1.2,1.2,1.0
